In [1]:
from pykrige.ok import OrdinaryKriging

In [2]:
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
from sklearn.model_selection import KFold

In [4]:
df = pd.read_csv("../data/beijinga_scaled.csv", index_col = 0)
# df = df.rename(columns={'ts': 'Time', 'station_id': 'Station'})

stations = df['station_id'].unique()
stations.sort()

splits = 6

kfout = KFold(n_splits=splits, shuffle=True, random_state=0)
kfin = KFold(n_splits=splits - 1, shuffle=True, random_state=0)

station_split = {}

for kout, (sts_ftrain_index, sts_test_index) in enumerate(kfout.split(stations)):
    station_split[kout] = {}
    for kin, (sts_train_index, sts_val_index) in enumerate(kfin.split(sts_ftrain_index)):
        sts_test = stations[sts_test_index]
        sts_val = stations[sts_ftrain_index[sts_val_index]]
        sts_train = stations[sts_ftrain_index[sts_train_index]]
        station_split[kout][kin] = {
            'test': sts_test,
            'train': sts_val,
            'pool': sts_train,
        }

In [5]:
from polire.interpolate import Kriging

In [14]:
df.head()

,latitude,longitude,ts,station_id,PM2.5
0,0.58292,0.17557,0.000000,1001.0,156.250000
0,0.58292,0.17557,0.002747,1001.0,24.833333
0,0.58292,0.17557,0.005495,1001.0,72.583333
0,0.58292,0.17557,0.008242,1001.0,30.043478
0,0.58292,0.17557,0.010989,1001.0,23.083333


In [15]:
times = df['ts'].unique()
times.sort()

In [16]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [17]:
errors = []
for kout in range(6):
    for kin in range(5):
        
        
        train_stations = list(station_split[kout][kin]['train'])
        pool_stations = list(station_split[kout][kin]['pool'])
        test_stations = list(station_split[kout][kin]['test'])

        
        test_df = df[df['station_id'].isin(test_stations)]
        pool_df = df[df['station_id'].isin(pool_stations)]
        train_df = df[df['station_id'].isin(train_stations)]


        print(kout, kin)
        
        test_error = {time_ix:np.nan for time_ix in range(29, 358)}
        


        for time_ix in range(29, 358):
            
            
            test_df = df[df['station_id'].isin(test_stations)]
            pool_df = df[df['station_id'].isin(pool_stations)]
            train_df = df[df['station_id'].isin(train_stations)]


            if time_ix % 30 == 0: # is this consitent with others? consistent.... hai
                pool_var = {}
                
                curr_timestamps = times[time_ix - 29:time_ix+1]
                curr_timestamps.sort()


                for time in curr_timestamps:



                    try:
                        temporal_train = train_df.groupby('ts').get_group(time)
                        temporal_pool = pool_df.groupby('ts').get_group(time)
                    except KeyError:
                        print("NOTHING")
                        continue
                    try:
                        krig = Kriging(require_variance=True)
                        X_train = np.array(temporal_train[['longitude', 'latitude']])
                        y_train = np.array(temporal_train[['PM2.5']])
                        krig.fit(X_train, y_train)
                        print ('good')
                                 
                    except Exception as e:
                        print(e)
                        continue
                    
                    
                    for station in pool_stations:
                        try:
                            pool_df = temporal_pool.groupby('station_id').get_group(station)
                        except:
                            print(station, "not present")
                            continue
                        X_pool_stn = np.array(pool_df[['longitude', 'latitude']])
                        mean = krig.predict(X_pool_stn)
                        var = krig.return_variance()
                        if station not in pool_var:
                            pool_var[station] = [var.data[0]]
                        else:
                            pool_var[station] = pool_var[station].append(var.data[0])
                for i in pool_var:
                    pool_var[i] = np.sum(pool_var[i]) / len(pool_var[i])
                    

                if len(pool_var) > 0:
                    station = max(pool_var, key=pool_var.get)
                    train_stations.append(station)
                    pool_stations.remove(station)
                else:
                    print("NO POOL VAR")

#             test_df = df[df['station_id'].isin(test_stations)]
#             pool_df = df[df['station_id'].isin(pool_stations)]
#             train_df = df[df['station_id'].isin(train_stations)]


            temporal_train_df = train_df[train_df['ts'] == times[time_ix]]
            temporal_test_df = test_df[test_df['ts']==times[time_ix]]


            X_train = np.array(temporal_train_df[['longitude', 'latitude']])
            y_train = np.array(temporal_train_df[['PM2.5']])


            X_test = np.array(temporal_test_df[['longitude', 'latitude']])
            y_test = np.array(temporal_test_df[['PM2.5']])

            if X_train.shape[0] > 1 and X_test.shape[0] > 1:
                
                try:
                    krig = Kriging()
                    krig.fit(X_train, y_train)

                    pred = krig.predict(X_test)

                    print(X_train.shape, len(train_stations), len(test_stations), len(pool_stations))

                    rmse = np.sqrt(mean_squared_error(pred, y_test))
                    test_error[time_ix] = [rmse]
                except Exception as e:
                    print(e)
            else:
                print("2 years waste")

            print(time_ix)


        errors.append(((kout, kin), pd.DataFrame(test_error).mean(axis = 1)))
#         break
#     break


0 0
(6, 2) 6 6 24
29
good
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
(5, 2) 7 6 23
30
(7, 2) 7 6 23
31
(7, 2) 7 6 23
32
(7, 2) 7 6 23
33
(7, 2) 7 6 23
34
(7, 2) 7 6 23
35
(7, 2) 7 6 23
36
(7, 2) 7 6 23
37
(7, 2) 7 6 23
38
(7, 2) 7 6 23
39
(7, 2) 7 6 23
40
(7, 2) 7 6 23
41
(7, 2) 7 6 23
42
(7, 2) 7 6 23
43
(7, 2) 7 6 23
44
(7, 2) 7 6 23
45
(6, 2) 7 6 23
46
(6, 2) 7 6 23
47
(6, 2) 7 6 23
48
(6, 2) 7 6 23
49
(6, 2) 7 6 23
50
(6, 2) 7 6 23
51
(6, 2) 7 6 23
52
(6, 2) 7 6 23
53
(6, 2) 7 6 23
54
(6, 2) 7 6 23
55
(6, 2) 7 6 23
56
(6, 2) 7 6 23
57
(6, 2) 7 6 23
58
(6, 2) 7 6 23
59
good
1019.0 not present
1027.0 not present
1036.0 not present
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NO

(16, 2) 16 6 14
329
good
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
(16, 2) 17 6 13
330
(17, 2) 17 6 13
331
(17, 2) 17 6 13
332
(17, 2) 17 6 13
333
(17, 2) 17 6 13
334
(17, 2) 17 6 13
335
(17, 2) 17 6 13
336
(17, 2) 17 6 13
337
(17, 2) 17 6 13
338
(17, 2) 17 6 13
339
(17, 2) 17 6 13
340
(17, 2) 17 6 13
341
(17, 2) 17 6 13
342
(17, 2) 17 6 13
343
(17, 2) 17 6 13
344
(17, 2) 17 6 13
345
(17, 2) 17 6 13
346
(17, 2) 17 6 13
347
(17, 2) 17 6 13
348
(17, 2) 17 6 13
349
(17, 2) 17 6 13
350
(17, 2) 17 6 13
351
(17, 2) 17 6 13
352
(17, 2) 17 6 13
353
(17, 2) 17 6 13
354
(17, 2) 17 6 13
355
(16, 2) 17 6 13
356
(16, 2) 17 6 13
357
0 1
(4, 2) 6 6 24
29
good
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHI

(15, 2) 15 6 15
289
(15, 2) 15 6 15
290
(15, 2) 15 6 15
291
(15, 2) 15 6 15
292
(15, 2) 15 6 15
293
(15, 2) 15 6 15
294
(15, 2) 15 6 15
295
(15, 2) 15 6 15
296
(15, 2) 15 6 15
297
(15, 2) 15 6 15
298
(15, 2) 15 6 15
299
good
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
(15, 2) 16 6 14
300
(16, 2) 16 6 14
301
(16, 2) 16 6 14
302
(16, 2) 16 6 14
303
(16, 2) 16 6 14
304
(16, 2) 16 6 14
305
(16, 2) 16 6 14
306
(16, 2) 16 6 14
307
(16, 2) 16 6 14
308
(16, 2) 16 6 14
309
(16, 2) 16 6 14
310
(15, 2) 16 6 14
311
(15, 2) 16 6 14
312
(15, 2) 16 6 14
313
(15, 2) 16 6 14
314
(16, 2) 16 6 14
315
(16, 2) 16 6 14
316
(16, 2) 16 6 14
317
(16, 2) 16 6 14
318
(16, 2) 16 6 14
319
(16, 2) 16 6 14
320
(16, 2) 16 6 14
321
(16, 2) 16 6 14
322
(16, 2) 16 6 14
323
(16, 2) 16 6 14
324
(16, 2) 16 6 14
325
(16, 2) 16 6 14
326
(16

/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/pykrige/core.py:565: RuntimeWarning: invalid value encountered in double_scalars
  (np.amax(lags) - np.amin(lags)), np.amin(semivariance)]
/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:371: RuntimeWarning: invalid value encountered in greater_equal
  return np.all((x >= lb) & (x <= ub))
/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/scipy/optimize/_lsq/common.py:371: RuntimeWarning: invalid value encountered in less_equal
  return np.all((x >= lb) & (x <= ub))


good
1008.0 not present
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
(9, 2) 10 6 20
120
(10, 2) 10 6 20
121
(10, 2) 10 6 20
122
(10, 2) 10 6 20
123
(10, 2) 10 6 20
124
(10, 2) 10 6 20
125
(10, 2) 10 6 20
126
(10, 2) 10 6 20
127
(10, 2) 10 6 20
128
(10, 2) 10 6 20
129
(10, 2) 10 6 20
130
(10, 2) 10 6 20
131
(10, 2) 10 6 20
132
(10, 2) 10 6 20
133
(10, 2) 10 6 20
134
(10, 2) 10 6 20
135
(10, 2) 10 6 20
136
(10, 2) 10 6 20
137
(10, 2) 10 6 20
138
(10, 2) 10 6 20
139
(10, 2) 10 6 20
140
(10, 2) 10 6 20
141
(10, 2) 10 6 20
142
(10, 2) 10 6 20
143
(10, 2) 10 6 20
144
(10, 2) 10 6 20
145
(10, 2) 10 6 20
146
(10, 2) 10 6 20
147
(10, 2) 10 6 20
148
(10, 2) 10 6 20
149
good
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTH

...

NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
(9, 2) 10 6 20
120
(9, 2) 10 6 20
121
(9, 2) 10 6 20
122
(10, 2) 10 6 20
123
(10, 2) 10 6 20
124
(10, 2) 10 6 20
125
(10, 2) 10 6 20
126
(10, 2) 10 6 20
127
(10, 2) 10 6 20
128
(10, 2) 10 6 20
129
(10, 2) 10 6 20
130
(10, 2) 10 6 20
131
(10, 2) 10 6 20
132
(10, 2) 10 6 20
133
(10, 2) 10 6 20
134
(9, 2) 10 6 20
135
(10, 2) 10 6 20
136
(10, 2) 10 6 20
137
(10, 2) 10 6 20
138
(10, 2) 10 6 20
139
(10, 2) 10 6 20
140
(10, 2) 10 6 20
141
(10, 2) 10 6 20
142
(10, 2) 10 6 20
143
(10, 2) 10 6 20
144
(10, 2) 10 6 20
145
(10, 2) 10 6 20
146
(10, 2) 10 6 20
147
(10, 2) 10 6 20
148
(10, 2) 10 6 20
149
good
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
(10, 2) 11 6 19
150
(11, 2) 11 6 19
151
(11, 2) 11 6 19
152
(10

NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
2 years waste
90
2 years waste
91
2 years waste
92
2 years waste
93
2 years waste
94
2 years waste
95
(9, 2) 9 6 21
96
(9, 2) 9 6 21
97
(9, 2) 9 6 21
98
(9, 2) 9 6 21
99
(9, 2) 9 6 21
100
(9, 2) 9 6 21
101
(9, 2) 9 6 21
102
(9, 2) 9 6 21
103
(9, 2) 9 6 21
104
(9, 2) 9 6 21
105
(9, 2) 9 6 21
106
(9, 2) 9 6 21
107
(9, 2) 9 6 21
108
(9, 2) 9 6 21
109
(9, 2) 9 6 21
110
(9, 2) 9 6 21
111
(9, 2) 9 6 21
112
(9, 2) 9 6 21
113
(9, 2) 9 6 21
114
(9, 2) 9 6 21
115
(9, 2) 9 6 21
116
(9, 2) 9 6 21
117
(9, 2) 9 6 21
118
(9, 2) 9 6 21
119
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
good
1008.0 not present
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
(9, 2) 10 6 20
120
(10, 2) 10 6 20
121
(10, 2) 10 6 20
122
(10, 2) 10 6 20
123
(10, 2) 10 6 20
124
(10, 2) 10 6 20
125
(10, 2) 10 6 20
126
(10, 2) 10 6 20
127

(8, 2) 8 6 22
64
(8, 2) 8 6 22
65
(8, 2) 8 6 22
66
(8, 2) 8 6 22
67
(8, 2) 8 6 22
68
(8, 2) 8 6 22
69
(8, 2) 8 6 22
70
(8, 2) 8 6 22
71
(8, 2) 8 6 22
72
(8, 2) 8 6 22
73
(8, 2) 8 6 22
74
(8, 2) 8 6 22
75
(8, 2) 8 6 22
76
(8, 2) 8 6 22
77
(8, 2) 8 6 22
78
(8, 2) 8 6 22
79
(8, 2) 8 6 22
80
(8, 2) 8 6 22
81
(8, 2) 8 6 22
82
(6, 2) 8 6 22
83
(7, 2) 8 6 22
84
(7, 2) 8 6 22
85
(8, 2) 8 6 22
86
(8, 2) 8 6 22
87
(8, 2) 8 6 22
88
2 years waste
89
good
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
2 years waste
90
2 years waste
91
2 years waste
92
2 years waste
93
2 years waste
94
2 years waste
95
(9, 2) 9 6 21
96
(9, 2) 9 6 21
97
(9, 2) 9 6 21
98
(9, 2) 9 6 21
99
(9, 2) 9 6 21
100
(9, 2) 9 6 21
101
(9, 2) 9 6 21
102
(9, 2) 9 6 21
103
(9, 2) 9 6 21
104
(9, 2) 9 6 21
105
(9, 2) 9 6 21
106
(9, 2) 9 6 21
107
(9, 2) 

(17, 2) 17 6 13
357
5 4
(4, 2) 6 6 24
29
good
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
(4, 2) 7 6 23
30
(4, 2) 7 6 23
31
(5, 2) 7 6 23
32
(6, 2) 7 6 23
33
(6, 2) 7 6 23
34
(6, 2) 7 6 23
35
(7, 2) 7 6 23
36
(7, 2) 7 6 23
37
(7, 2) 7 6 23
38
(7, 2) 7 6 23
39
(7, 2) 7 6 23
40
(7, 2) 7 6 23
41
(7, 2) 7 6 23
42
(7, 2) 7 6 23
43
(7, 2) 7 6 23
44
(7, 2) 7 6 23
45
(7, 2) 7 6 23
46
(7, 2) 7 6 23
47
(7, 2) 7 6 23
48
(7, 2) 7 6 23
49
(7, 2) 7 6 23
50
(7, 2) 7 6 23
51
(7, 2) 7 6 23
52
(6, 2) 7 6 23
53
(7, 2) 7 6 23
54
(7, 2) 7 6 23
55
(7, 2) 7 6 23
56
(7, 2) 7 6 23
57
(7, 2) 7 6 23
58
(7, 2) 7 6 23
59
good
1032.0 not present
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTH

good
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
NOTHING
(16, 2) 17 6 13
330
(17, 2) 17 6 13
331
(17, 2) 17 6 13
332
(17, 2) 17 6 13
333
(17, 2) 17 6 13
334
(17, 2) 17 6 13
335
(17, 2) 17 6 13
336
(17, 2) 17 6 13
337
(17, 2) 17 6 13
338
(17, 2) 17 6 13
339
(17, 2) 17 6 13
340
(17, 2) 17 6 13
341
(17, 2) 17 6 13
342
(17, 2) 17 6 13
343
(17, 2) 17 6 13
344
(17, 2) 17 6 13
345
(17, 2) 17 6 13
346
(17, 2) 17 6 13
347
(17, 2) 17 6 13
348
(17, 2) 17 6 13
349
(16, 2) 17 6 13
350
(17, 2) 17 6 13
351
(17, 2) 17 6 13
352
(17, 2) 17 6 13
353
(17, 2) 17 6 13
354
(17, 2) 17 6 13
355
(17, 2) 17 6 13
356
(17, 2) 17 6 13
357


In [18]:
len(errors)

30

In [35]:
d = {}
for e in errors:
    temp = d.get('kout', [])
    temp.append(e[0][0])
    d['kout'] = temp
    
    temp = d.get('kin', [])
    temp.append(e[0][1])
    d['kin'] = temp
    
    temp = d.get('rmse', [])
    temp.append(e[1][0])
    d['rmse'] = temp
    

In [37]:
pd.DataFrame(d).to_csv('active_kriging_results.csv', index=None)

In [26]:
np.mean(errors)


16.491531346446706

In [30]:
for i in errors:
    print(i)

0    15.107037
dtype: float64
0    15.960798
dtype: float64
0    16.52857
dtype: float64
0    15.294778
dtype: float64
0    16.662367
dtype: float64
0    19.603927
dtype: float64
0    19.309057
dtype: float64
0    18.156012
dtype: float64
0    20.53913
dtype: float64
0    18.578064
dtype: float64
0    16.818201
dtype: float64
0    17.460462
dtype: float64
0    17.72717
dtype: float64
0    19.369227
dtype: float64
0    17.544565
dtype: float64
0    14.902887
dtype: float64
0    13.07482
dtype: float64
0    12.717138
dtype: float64
0    12.693101
dtype: float64
0    14.364689
dtype: float64
0    16.10218
dtype: float64
0    17.280655
dtype: float64
0    15.110234
dtype: float64
0    16.533254
dtype: float64
0    15.30872
dtype: float64
0    16.460181
dtype: float64
0    17.037894
dtype: float64
0    15.565741
dtype: float64
0    16.250929
dtype: float64
0    16.684153
dtype: float64


In [ ]:
temporal_train

In [42]:
np.round(df["latitude"].unique(), 3).shape

(92,)

In [38]:
np.round?